In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pickle
import statistics
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import ndcg_score
from sklearn.metrics import f1_score

In [3]:
user_grouped = pd.read_csv('/content/drive/MyDrive/Web Mining/Pipeline/users_grouped_by3.csv')
cb_predictions=pickle.load( open( "/content/drive/MyDrive/Web Mining/Pipeline/cb_predictions.p", "rb" ) )
cf_predictions=pickle.load( open( "/content/drive/MyDrive/Web Mining/Pipeline/cf_predictions.p", "rb" ) )
gs=pickle.load( open( "/content/drive/MyDrive/Web Mining/Pipeline/true_value.p", "rb" ) )
# cb_predictions==cf_predictions

In [4]:
def evaluate(true_values,predictions,t):
# do aggregation 
# Average 
  if t=="average":
    yp=np.sum(predictions, axis=0)
    yp=yp/len(predictions)
    yt=np.sum(true_values, axis=0)
    yt=yt/len(true_values)
  elif t=="least misery":
    yp=np.amin(predictions, axis=0)
    yt=np.amin(true_values, axis=0)
  elif t=="most pleasure":
    yp=np.amax(predictions, axis=0)
    yt=np.amax(true_values, axis=0)
  elif t=="average without misery":
    yp=np.sum(predictions, axis=0)
    min=np.amin(predictions, axis=0)
    yp=yp-min
    yp=yp/(len(predictions)-1)
    yt=np.sum(true_values, axis=0)
    min=np.amin(true_values, axis=0)
    yt=yt-min
    yt=yt/(len(true_values)-1)
  else: 
    print("only these options available: [average,least misery,most pleasure,average without misery]")
  # if the score is more than 3 then he likes the movie
  yt=np.where(yt < 4, 0, 1)
  yp=np.where(yp < 4, 0, 1)
  return accuracy_score(np.round(yt),np.round(yp)),f1_score(np.round(yt),np.round(yp))
def get_predections(true_values,predictions,t):
# do aggregation 
# Average 
  if t=="average":
    yp=np.sum(predictions, axis=0)
    yp=yp/len(predictions)
    yt=np.sum(true_values, axis=0)
    yt=yt/len(true_values)
  elif t=="least misery":
    yp=np.amin(predictions, axis=0)
    yt=np.amin(true_values, axis=0)
  elif t=="most pleasure":
    yp=np.amax(predictions, axis=0)
    yt=np.amax(true_values, axis=0)
  elif t=="average without misery":
    yp=np.sum(predictions, axis=0)
    min=np.amin(predictions, axis=0)
    yp=yp-min
    yp=yp/(len(predictions)-1)
    yt=np.sum(true_values, axis=0)
    min=np.amin(true_values, axis=0)
    yt=yt-min
    yt=yt/(len(true_values)-1)
  else: 
    print("only these options available: [average,least misery,most pleasure,average without misery]")
  
  return yt,yp
def hybrid_evaluate(yt,predictions1,predictions2,t):
# do aggregation 
# Average 
  if t=="average":
    yp=(predictions1+predictions2)/2
  elif t=="least misery":
    yp=np.amin(np.array([predictions1,predictions2]), axis=0)
  elif t=="most pleasure":
    yp=np.amax(np.array([predictions1,predictions2]), axis=0)
  elif t=="average without misery":
    yp=np.sum(np.array([predictions1,predictions2]), axis=0)
    min=np.amin(np.array([predictions1,predictions2]), axis=0)
    yp=yp-min
  else: 
    print("only these options available: [average,least misery,most pleasure,average without misery]")
  # if the score is more than 3 then he likes the movie
  yt=np.where(yt < 4, 0, 1)
  yp=np.where(yp < 4, 0, 1)
  return accuracy_score(np.round(yt),np.round(yp)),f1_score(np.round(yt),np.round(yp))

In [23]:
def group_eval(t):
  results1_acc=[]
  results1_ndcg=[]
  results2_acc=[]
  results2_ndcg=[]
  groupsID=user_grouped['Group'].unique()
  for g in groupsID:
    results=evaluate(gs[g],cb_predictions[g],t)
    results1_acc.append(results[0])
    results1_ndcg.append(results[1])
    results=evaluate(gs[g],cf_predictions[g],t)
    results2_acc.append(results[0])
    results2_ndcg.append(results[1])
  print("Content based",t,round(statistics.mean(results1_acc),4)*100)
  print("Content based",t,round(statistics.mean(results1_ndcg),4)*100)
  print("Collaborative ",t,round(statistics.mean(results2_acc),4)*100)
  print("Collaborative ",t,round(statistics.mean(results2_ndcg),4)*100)
def hybrid_eval(t,t2):
  results2_acc=[]
  results2_ndcg=[]
  groupsID=user_grouped['Group'].unique()
  for g in groupsID:
    yp1,yt=get_predections(gs[g],cb_predictions[g],t2)
    yp2,yt=get_predections(gs[g],cf_predictions[g],t2)
    results=hybrid_evaluate(yt,yp1,yp2,t)
    results2_acc.append(results[0])
    results2_ndcg.append(results[1])
  print("hybrid ",t,t2,statistics.mean(results2_acc))
  print("hybrid ",t,t2,statistics.mean(results2_ndcg))

# Evaluate each model, based on accuarcy and F1 score 

In [24]:
group_eval('average')
group_eval('least misery')
group_eval('most pleasure')
group_eval('average without misery')

Content based average 57.48
Content based average 2.7
Collaborative  average 57.98
Collaborative  average 3.2399999999999998
Content based least misery 73.58
Content based least misery 0.7000000000000001
Collaborative  least misery 73.68
Collaborative  least misery 0.27
Content based most pleasure 19.66
Content based most pleasure 16.82
Collaborative  most pleasure 30.459999999999997
Collaborative  most pleasure 35.14
Content based average without misery 32.85
Content based average without misery 6.49
Collaborative  average without misery 35.57
Collaborative  average without misery 13.600000000000001


# Evaluate the hybrid model

In [16]:
m=['average','least misery','most pleasure','average without misery']
for t2 in m:
  hybrid_eval(m[0],t2)
  hybrid_eval(m[1],t2)
  hybrid_eval(m[2],t2)
  hybrid_eval(m[3],t2)

hybrid  average average 0.5797557695888073
hybrid  average average 0.032414427631445204
hybrid  least misery average 0.5797557695888073
hybrid  least misery average 0.032414427631445204
hybrid  most pleasure average 0.5797557695888073
hybrid  most pleasure average 0.032414427631445204
hybrid  average without misery average 0.5797557695888073
hybrid  average without misery average 0.032414427631445204
hybrid  average least misery 0.7368163386498936
hybrid  average least misery 0.00274636812796454
hybrid  least misery least misery 0.7368163386498936
hybrid  least misery least misery 0.00274636812796454
hybrid  most pleasure least misery 0.7368163386498936
hybrid  most pleasure least misery 0.00274636812796454
hybrid  average without misery least misery 0.7368163386498936
hybrid  average without misery least misery 0.00274636812796454
hybrid  average most pleasure 0.30462793440909736
hybrid  average most pleasure 0.35135964574535167
hybrid  least misery most pleasure 0.30462793440909736
h